# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [29]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [30]:
from apisecrets import API_KEY

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [31]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [32]:
symbol = "aapl"
api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={API_KEY}'
data = requests.get(api_url).json()
print(data)

[{'avgTotalVolume': 64672410, 'calculationPrice': 'close', 'change': 0.9, 'changePercent': 0.0055, 'close': 164.66, 'closeSource': 'official', 'closeTime': 1680811200252, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 164.63, 'delayedPriceTime': 1680811181490, 'extendedChange': -0.09, 'extendedChangePercent': -0.00055, 'extendedPrice': 164.57, 'extendedPriceTime': 1680825599748, 'high': 164.66, 'highSource': 'Close', 'highTime': 1680811200252, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 164.66, 'iexCloseTime': 1680811199902, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 162.37, 'iexOpenTime': 1680787801037, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1680811199940, 'latestPrice': 164.66, 'latestSource': 'Close', 'latestTime': 'April 6, 2023', 'latestUpdate': 1680811200252, 'latestVolume': 45390123, 'low': 164.66, 'lowSource': 'Close', 'lowTime': 1680811200000, 'ma

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [33]:
price = data[0]['latestPrice']
marketCap = data[0]['marketCap']
print(price, marketCap)

164.66 2605241628360


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [34]:
my_columns = [ 'Ticker', 'Stock Price', 'Market Cap', 'Number of Shares to buy']
final_dataFrame = pd.DataFrame(columns = my_columns)
final_dataFrame

,Ticker,Stock Price,Market Cap,Number of Shares to buy


In [35]:
final_dataFrame = pd.concat([final_dataFrame, pd.DataFrame.from_records([{ 'Ticker' : symbol, 'Stock Price' : price, 'Market Cap' : marketCap, 'Number of Shares to buy' : 'N/A' }])])

final_dataFrame

,Ticker,Stock Price,Market Cap,Number of Shares to buy
0,aapl,164.66,2605241628360,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [36]:
# final_dataFrame = pd.DataFrame(columns = my_columns)
# for stock in stocks['Ticker']:
#     api_url = f'https://api.iex.cloud/v1/data/core/quote/{stock}?token={API_KEY}'
#     data = requests.get(api_url).json()
#     final_dataFrame = pd.concat([final_dataFrame, pd.DataFrame.from_records([{ 'Ticker' : stock, 'Stock Price' : data[0]['latestPrice'], 'Market Cap' : data[0]['marketCap'], 'Number of Shares to buy' : 'N/A' }])])

In [37]:
final_dataFrame

,Ticker,Stock Price,Market Cap,Number of Shares to buy
0,aapl,164.66,2605241628360,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [38]:
def chunks(lst, n):
    yield [lst[x:x+n] for x in range(0, len(lst), n)]

In [39]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups[0])):
    symbol_strings.append(','.join(symbol_groups[0][i]))
    # print(symbol_strings[i])

final_dataFrame = pd.DataFrame(columns = my_columns)

# for symbol_string in symbol_strings[:1]:
#     batch_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token={API_KEY}'
#     data = requests.get(batch_url).json()
#     for symbol in symbol_string.split(',')[:1]:
#         print(symbol)
#         print(data[0])

for symbol_string in symbol_strings:
    batch_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token={API_KEY}'
    data = requests.get(batch_url).json()
    for stock in data:
        final_dataFrame = pd.concat([final_dataFrame, pd.DataFrame.from_records([{ 'Ticker' : stock['symbol'], 'Stock Price' : stock['latestPrice'], 'Market Cap' : stock['marketCap'], 'Number of Shares to buy' : 'N/A' }])], ignore_index=True)
    
final_dataFrame

,Ticker,Stock Price,Market Cap,Number of Shares to buy
0,A,138.03,40815726770,N/A
1,AAL,14.01,9145959039,N/A
2,AAP,120.69,7153752629,N/A
3,AAPL,164.66,2605241628360,N/A
4,ABBV,161.55,284997105690,N/A
...,...,...,...,...
500,YUM,132.10,37002248702,N/A
501,ZBH,129.17,26993979668,N/A
502,ZBRA,300.26,15433166129,N/A
503,ZION,29.39,4352631315,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [40]:
portfolio_size = input('Enter the value of the portfolio:')

try:
    portfolio_size = float(portfolio_size)
except ValueError:
    portfolio_size = input('Enter the value of the portfolio:')
    portfolio_size = float(portfolio_size)
portfolio_size

10000000.0

In [41]:
position_size = portfolio_size / len(final_dataFrame.index)
for i in range(0, len(final_dataFrame.index)):
    final_dataFrame.loc[i, 'Number of Shares to buy'] = math.floor(position_size / final_dataFrame.loc[i, 'Stock Price'])

final_dataFrame

,Ticker,Stock Price,Market Cap,Number of Shares to buy
0,A,138.03,40815726770,143
1,AAL,14.01,9145959039,1413
2,AAP,120.69,7153752629,164
3,AAPL,164.66,2605241628360,120
4,ABBV,161.55,284997105690,122
...,...,...,...,...
500,YUM,132.10,37002248702,149
501,ZBH,129.17,26993979668,153
502,ZBRA,300.26,15433166129,65
503,ZION,29.39,4352631315,673


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [42]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
final_dataFrame.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [43]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format({
    'font_color' : font_color,
    'bg_color' : background_color,
    'border' : 1
})

dollar_format = writer.book.add_format({
    'num_format' : '$0.00',
    'font_color' : font_color,
    'bg_color' : background_color,
    'border' : 1
})

integer_format = writer.book.add_format({
    'num_format' : '0',
    'font_color' : font_color,
    'bg_color' : background_color,
    'border' : 1
})

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [44]:
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [45]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [46]:
writer.save()

C:\Users\suren\AppData\Local\Temp\ipykernel_12836\934276808.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
